In [2]:
# coding: utf-8
"""
ファイルの編集を色々とするためのファイル
"""
from filer2.filer2 import Filer
import numpy as np
import glob
import collections
import re
import MeCab
import itertools

In [3]:
# 関数の定義
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

def morpho(sentence):
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if (features[0] == "名詞" and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]) or features[0] == "形容詞":
            if features[6] == "*":
                if res.surface != '':
                    list_words.append(res.surface)
            else:
                if features[6] != '':
                    list_words.append(features[6])
        res = res.next
    return list_words

def morpho1(sentence):
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if features[0] != "記号":
            if features[6] == "*":
                if res.surface != '':
                    list_words.append(res.surface)
            else:
                if features[6] != '':
                    list_words.append(features[6])
        res = res.next
    return list_words

### 全体のコーパスを施設別に集計する

In [4]:
list_filepath = glob.glob('../topic_model/files/rakuten_corpus/rakuten_corpus_master/txtfile/*')

In [5]:
# 読み込み
list_file = [Filer.readtxt(path)  for path in list_filepath]

In [12]:
# タブで区切る
list_file = [[sentence.split('\t') for sentence in row] for row in list_file]

In [53]:
# ホテル番号別に集計
dict_hotel_sentence = collections.defaultdict(list)
for row in list_file:
    for sen in row:
        dict_hotel_sentence[sen[0]].append(sen[1] + '\t' + sen[2])

In [54]:
# review数が100件未満のホテルは除外して保存する
for key, value in dict_hotel_sentence.items():
    if len(value) >= 100:
        Filer.writetxt(value, '../files/txtfile/review-'+key+'.txt')

In [87]:
# 数を集計
dict_count = {key: len(value) for key, value in dict_hotel_sentence.items()}

In [50]:
# 施設名と番号を合わせる
list_name = Filer.readtxt('../files/travel03_hotelMaster_20160304.txt')
list_name = [sen.split('\t') for sen in list_name]
dict_num_name = {row[0]: row[1] for row in list_name if len(row) == 2}

In [86]:
# 年別にレビュー数を集計
dict_counter = {}
for key, value in dict_hotel_sentence.items():
    dict_counter_tmp = collections.defaultdict(int)
    for sen in value:
        dict_counter_tmp[sen[0:4]] += 1
    dict_counter[key] = dict_counter_tmp

In [84]:
dict_count['9680']

defaultdict(int,
            {'2001': 3,
             '2002': 6,
             '2003': 17,
             '2004': 6,
             '2005': 54,
             '2006': 288,
             '2007': 96,
             '2008': 42,
             '2009': 38,
             '2010': 31,
             '2011': 11})

In [90]:
# 施設番号、施設名、レビュー数のファイルを作成する
list_master = [['施設番号', '施設名', 'レビュー数',
                '1999', '2000', '2001', '2002',
                '2003', '2004', '2005', '2006',
                '2007', '2008', '2009', '2010',
                '2011']]
for key, value in sorted(dict_count.items(), key=lambda x:x[1], reverse=True):
    if key in dict_num_name:
        list_master.append([key, dict_num_name[key], value, dict_counter[key]['1999'],
                            dict_counter[key]['2000'], dict_counter[key]['2001'], dict_counter[key]['2002'],
                            dict_counter[key]['2003'], dict_counter[key]['2004'], dict_counter[key]['2005'],
                            dict_counter[key]['2006'], dict_counter[key]['2007'], dict_counter[key]['2008'],
                            dict_counter[key]['2009'], dict_counter[key]['2010'], dict_counter[key]['2011']])
    else:
        list_master.append([key, '', value, dict_counter[key]['1999'],
                            dict_counter[key]['2000'], dict_counter[key]['2001'], dict_counter[key]['2002'],
                            dict_counter[key]['2003'], dict_counter[key]['2004'], dict_counter[key]['2005'],
                            dict_counter[key]['2006'], dict_counter[key]['2007'], dict_counter[key]['2008'],
                            dict_counter[key]['2009'], dict_counter[key]['2010'], dict_counter[key]['2011']])

In [91]:
# csvfileとして保存
Filer.writecsv(list_master, '../files/txtfile/master.csv')

### 分析対象とするホテルのみを取り出して、sentenceごとのファイルにする
* 51637：アパホテル＆リゾート＜東京ベイ幕張＞
* 1238：オリエンタルホテル東京ベイ
* 53171：加賀の湧泉　ドーミーイン金沢
* 19191：新横浜プリンスホテル
* 76887：ホテルモントレ　グラスミア大阪

In [92]:
list_filepath = ['../files/txtfile/review-51637.txt',
                 '../files/txtfile/review-1238.txt',
                 '../files/txtfile/review-53171.txt',
                 '../files/txtfile/review-19191.txt',
                 '../files/txtfile/review-76887.txt',]

In [93]:
# 2010年、2011年の文章のみを取得
list_review_rev = []
for path in list_filepath:
    list_review = Filer.readtxt(path)
    list_tmp = []
    for review in list_review:
        if review[0:4] == '2010' or review[0:4] == '2011':
            list_tmp.append(review.split('\t')[1])
    list_review_rev.append(list_tmp)

In [103]:
#【ご利用の宿泊プラン】~を削除
pattern = r'【ご利用の宿泊プラン】.+$'
list_review_rev1 = []
for row in list_review_rev:
    list_tmp = []
    for sentence in row:
        list_tmp.append(re.sub(pattern, '', sentence))
    list_review_rev1.append(list_tmp)

In [164]:
# sentenceに変更、その際、ユーザー番号を頭に振る
list_sentence = []
for row in list_review_rev1:
    list_tmp = []
    for i, review in enumerate(row):
        list_s = re.split(r'。|！|？', review)
        for sen in list_s:
            list_tmp.append([i, sen])
    list_sentence.append(list_tmp)

In [165]:
# 文字の前後の空白を削除
list_sentence_rev = []
for row in list_sentence:
    list_tmp = []
    for sen in row:
        sentence = sen[1].replace(' ', '')
        if len(sentence) > 1:
            list_tmp.append([sen[0], sentence])
    list_sentence_rev.append(list_tmp)

In [185]:
# 先頭が、」もしくは）で始まってる場合、前の文にくっつける
list_sentence_rev1 = []
for row in list_sentence_rev:
    list_tmp = []
    for i in range(len(row)-1):
        if re.match(r'」|』|\)|）', row[i][1]):
            pass
        elif re.match(r'」|』|\)|）', row[i+1][1]):
            list_tmp.append([row[i][0], row[i][1]+row[i+1][1]])
        else:
            list_tmp.append([row[i][0], row[i][1]])
    else:
        list_tmp.append([row[i+1][0], row[i+1][1]])
        list_sentence_rev1.append(list_tmp)

In [187]:
Filer.writetsv(list_sentence_rev1[0], '../files/sentencefile/sentence-51637.tsv')
Filer.writetsv(list_sentence_rev1[1], '../files/sentencefile/sentence-1238.tsv')
Filer.writetsv(list_sentence_rev1[2], '../files/sentencefile/sentence-53171.tsv')
Filer.writetsv(list_sentence_rev1[3], '../files/sentencefile/sentence-19191.tsv')
Filer.writetsv(list_sentence_rev1[4], '../files/sentencefile/sentence-76887.tsv')

### sentenceごとのファイルを形態素解析済みのファイルに変更する

In [188]:
list_path = glob.glob('../files/sentencefile/*')
print list_path

['../files/sentencefile/sentence-76887.tsv', '../files/sentencefile/sentence-1238.tsv', '../files/sentencefile/sentence-53171.tsv', '../files/sentencefile/sentence-19191.tsv', '../files/sentencefile/sentence-51637.tsv']


In [189]:
list_sentence = [Filer.readtsv(path) for path in list_path]

In [198]:
list_morpho = [[[sen[1], morpho(sen[1])] for sen in row]for row in list_sentence]

In [209]:
Filer.writedump(list_morpho[4], '../files/morphofile/type1/morpho-51637.dump')

In [210]:
list_morpho1 = [[[sen[1], morpho1(sen[1])] for sen in row] for row in list_sentence]

In [216]:
Filer.writedump(list_morpho1[4], '../files/morphofile/type2/morpho-51637.dump')

### 高頻度語、低頻度語を数え上げて削除したファイルを作成する

In [218]:
list_word = [word for row in list_morpho for sen in row for word in sen[1]]
list_word1 = [word for row in list_morpho1 for sen in row for word in sen[1]]
dict_word_freq = collections.Counter(list_word)
dict_word_freq1 = collections.Counter(list_word1)
list_set_word = []
for row in list_morpho:
    for sen in row:
        list_set_word.extend(list(set(sen[1])))
        
list_set_word1 = []
for row in list_morpho1:
    for sen in row:
        list_set_word1.extend(list(set(sen[1])))

dict_document_freq = collections.Counter(list_set_word)
dict_document_freq1 = collections.Counter(list_set_word1)
print len(list_morpho[0])+len(list_morpho[1])+len(list_morpho[2])+len(list_morpho[3])+len(list_morpho[4])
print len(list_morpho1[0])+len(list_morpho1[1])+len(list_morpho1[2])+len(list_morpho1[3])+len(list_morpho1[4])

In [244]:
# 頻度の確認、10%切りで良さそう
for row in sorted(dict_document_freq.items(), key=lambda x:x[1], reverse=True)[0:10]:
    print row[0], row[1]

部屋 0.166852010265
利用 0.136783575706
ホテル 0.100427715997
良い 0.0860992301112
宿泊 0.0706159110351
満足 0.0563301967494
便利 0.047005988024
朝食 0.0458083832335
よい 0.0455089820359
広い 0.0444824636441


In [245]:
# 除去語の記録、10%以上、2回以下
list_remove = []
for row in dict_document_freq.items():
    if row[1] >= 0.1:
        list_remove.append(row[0])
for row in dict_word_freq.items():
    if row[1] <= 2:
        list_remove.append(row[0])

In [262]:
Filer.writetxt(list_remove, '../files/preprocessedfile/dict/remove_words.txt')

In [249]:
list_path = glob.glob('../files/morphofile/type1/*.dump')
list_path

['../files/morphofile/type1/morpho-51637.dump',
 '../files/morphofile/type1/morpho-53171.dump',
 '../files/morphofile/type1/morpho-1238.dump',
 '../files/morphofile/type1/morpho-76887.dump',
 '../files/morphofile/type1/morpho-19191.dump']

In [250]:
list_sentence = [Filer.readdump(path) for path in list_path]
list_sentence_rev = []
for row in list_sentence:
    list_tmp = []
    for sen in row:
        list_tmp_tmp = [word for word in sen[1] if word not in list_remove]
        list_tmp.append([sen[0], list_tmp_tmp])
    list_sentence_rev.append(list_tmp)

In [258]:
# 総単語数が１以下の文を削除
list_sentence_rev = [[sen for sen in row if len(sen[1]) >= 2] for row in list_sentence_rev]

In [267]:
Filer.writedump(list_sentence_rev[4], '../files/preprocessedfile/type11/wordlist/preprocessed-19191.dump')

### エッジリストの作成

In [268]:
list_path = glob.glob('../files/preprocessedfile/type11/wordlist/*')
list_path

['../files/preprocessedfile/type11/wordlist/preprocessed-19191.dump',
 '../files/preprocessedfile/type11/wordlist/preprocessed-76887.dump',
 '../files/preprocessedfile/type11/wordlist/preprocessed-1238.dump',
 '../files/preprocessedfile/type11/wordlist/preprocessed-51637.dump',
 '../files/preprocessedfile/type11/wordlist/preprocessed-53171.dump']

In [269]:
list_sentence = [Filer.readdump(path) for path in list_path]

In [270]:
# bigram
list_bigram = []
for row in list_sentence:
    list_tmp = []
    for sen in row:
        for i in range(len(sen[1])-1):
            list_tmp.append([sen[1][i], sen[1][i+1]])
    list_bigram.append(list_tmp)

In [279]:
Filer.writetsv(list_bigram[4], '../files/preprocessedfile/type11/bigram/bigram-53171.tsv')

In [322]:
# cor
list_cor = []
for row in list_sentence:
    list_tmp = []
    for sen in row:
        list_tmp.extend(list(itertools.combinations(tuple(sen[1]),2)))
    list_tmp = [list(row) for row in list_tmp]
    list_cor.append(list_tmp)

In [330]:
Filer.writetsv(list_cor[4], '../files/preprocessedfile/type11/cor/cor-53171.tsv')

In [311]:
seq = ['a', 'b', 'c', 'd', 'e']
list(itertools.combinations(tuple(seq),2))

[('a', 'b'),
 ('a', 'c'),
 ('a', 'd'),
 ('a', 'e'),
 ('b', 'c'),
 ('b', 'd'),
 ('b', 'e'),
 ('c', 'd'),
 ('c', 'e'),
 ('d', 'e')]

### PRTMによって、分類した後の文書を形態素解析する

In [9]:
list_path = glob.glob('classifiedfile/type11/bigram/sentence/*')
removeword = 'classifiedfile/type11/bigram/sentence/classified'
list_name = [path.replace(removeword, '') for path in list_path]
list_name = [name.replace('.txt', '') for name in list_name]

In [11]:
# 読み込み
list_sentence = [Filer.readtxt(path) for path in list_path]

In [12]:
# 形態素解析
list_master = []
for row in list_sentence:
    list_tmp = [morpho(sentence) for sentence in row]
    list_master.append(list_tmp)

In [24]:
# 保存
for row, name in zip(list_master, list_name):
    Filer.writetsv(row, './classifiedfile/type11/bigram/morpho/classified'+name+'.tsv')

In [25]:
# エッジリストの作成
list_bigram = []
for row in list_master:
    list_tmp = []
    for sen in row:
        for i in range(len(sen)-1):
            list_tmp.append([sen[i], sen[i+1]])
    list_bigram.append(list_tmp)

In [28]:
# 保存
for row, name in zip(list_bigram, list_name):
    Filer.writetsv(row, './classifiedfile/type11/bigram/bigram/classified'+name+'.tsv')

In [29]:
# cor
list_cor = []
for row in list_master:
    list_tmp = []
    for sen in row:
        list_tmp.extend(list(itertools.combinations(tuple(sen),2)))
    list_tmp = [list(row) for row in list_tmp]
    list_cor.append(list_tmp)

In [33]:
# 保存
for row, name in zip(list_cor, list_name):
    Filer.writetsv(row, './classifiedfile/type11/bigram/cor/classified'+name+'.tsv')